In [14]:
import random
import numba
import numpy as np

In [11]:

def monte_carlo_pi(nsamples=100):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4 * acc / nsamples

%timeit monte_carlo_pi(1_000_000)

375 ms ± 26.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
@numba.njit
def jitted_monte_carlo_pi(nsamples=100):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4 * acc / nsamples


%timeit jitted_monte_carlo_pi(1_000_000)

9.35 ms ± 261 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
x = np.random.random(1_000_000)
y = np.random.random(1_000_000)

In [17]:
def monte_carlo_pi_np(x, y):
    acc = np.sum(x**2 + y**2 < 1)

    return 4 * acc / len(x)


%timeit monte_carlo_pi_np(x, y)

10.8 ms ± 5.26 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
@numba.njit
def monte_carlo_pi_np_jitted(x, y):
    acc = np.sum(
        x ** 2 + y ** 2 < 1
    )

    return 4 * acc / len(x)

%timeit monte_carlo_pi_np_jitted(x, y)



2.8 ms ± 940 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
import numba as nb
from numba import types, typed
from numba.experimental import jitclass
from abc import ABC, abstractmethod

# key and value types
kv_ty = (types.int64, types.unicode_type)


# A container class with:
# * member 'd' holding a typed dictionary of int64 -> unicode string (kv_ty)
# * member 'l' holding a typed list of float64
@jitclass([("d", types.DictType(*kv_ty)), ("l", types.ListType(types.float64))])
class ContainerHolder(object):
    def __init__(self):
        # initialize the containers
        self.d = typed.Dict.empty(*kv_ty)
        self.l = typed.List.empty_list(types.float64)

    def alli(self):
        return self.d


container = ContainerHolder()
container.d[1] = "apple"
container.d[2] = "orange"
container.l.append(123.0)
container.l.append(456.0)
print(container.d)  # {1: apple, 2: orange}
print(container.l)  # [123.0, 456.0]

{1: apple, 2: orange}
[123.0, 456.0, ...]


In [28]:
abstract_spec = [("_data", types.ListType(types.Opaque("object")))]


@jitclass(abstract_spec)
class AbstractSerialMapper(ABC):
    def __init__(self):
        self._data = []

    def __len__(self):
        return len(self._data)

    def __getitem__(self, key):
        return self._data[key]

    def __iter__(self):
        return iter(range(len(self._data)))

TypeError: class members are not yet supported: __slots__, __abstractmethods__, _abc_impl

In [29]:
import numba as nb
from numba.experimental import jitclass


# =============================================================================
# Definition of Point
# =============================================================================
@jitclass([("x", nb.f8), ("y", nb.f8)])
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y


typePoint = Point.class_type.instance_type

# Example
p = Point(0.0, 0.0)
print(f"Point => x:{p.x}, y:{p.y}")
print(Point(0.0, 0.0) == Point(0.0, 0.0))
print(Point(0.0, 0.0) == Point(1.0, 0.0))
# Point => x:0.0, y:0.0
# True
# False


# =============================================================================
# Definition of Edge
# =============================================================================
@jitclass(
    [
        ("id", nb.i8),
        ("p0", typePoint),
        ("p1", typePoint),
        ("next", nb.i8),
        ("inv", nb.i8),
        ("triangle", nb.i8),
    ]
)
class Edge:
    def __init__(self, id: int, p0, p1):
        self.id = id
        self.p0 = p0
        self.p1 = p1
        self.next = -1
        self.inv = -1
        self.triangle = -1


typeEdge = Edge.class_type.instance_type

# Example
edge = Edge(1, Point(1.0, 0.0), Point(0.0, 1.0))
print(
    f"\nEdge => id: {edge.id}, p0: ({edge.p0.x}, {edge.p0.y}), p1: ({edge.p1.x}, {edge.p1.y})"
)


# =============================================================================
# Definition of Triangle
# =============================================================================
@jitclass(
    [
        ("e01", typeEdge),
        ("e12", typeEdge),
        ("e20", typeEdge),
        ("id", nb.u8),
        ("p0", typePoint),
        ("p1", typePoint),
        ("p2", typePoint),
        ("vertices", nb.types.ListType(typePoint)),
    ]
)
class Triangle:
    def __init__(self, id: int, e01, e12, e20):
        self.id = id
        self.e01 = e01
        self.e12 = e12
        self.e20 = e20
        self.e01.next = e12.id
        self.e12.next = e20.id
        self.e20.next = e01.id
        self.e01.triangle = id
        self.e12.triangle = id
        self.e20.triangle = id
        self.p0 = e01.p0
        self.p1 = e01.p1
        self.p2 = e12.p1
        self.vertices = nb.typed.List([self.p0, self.p1, self.p2])
        if self.closed():
            print("I am a triangle.")
        else:
            print("I am not a triangle.")

    def closed(self) -> bool:
        return (
            (self.e01.p1 == self.e12.p0)
            and (self.e12.p1 == self.e20.p0)
            and (self.e20.p1 == self.e01.p0)
        )


typeTriangle = Triangle.class_type.instance_type

# Example 1 (closed)
p0 = Point(0, 0)
p1 = Point(1, 0)
p2 = Point(0, 1)
print()
t = Triangle(1, Edge(1, p0, p1), Edge(2, p1, p2), Edge(3, p2, p0))
print(f"Triangle => id: {t.id} (closed)")
for v in t.vertices:
    print(f"            x:{v.x}, y:{v.y}")
print(f"    Is triangle closed? => {t.closed()}")

# Example 2 (not closed)
p0 = Point(0, 0)
p1 = Point(1, 0)
p2 = Point(0, 1)
print()
t = Triangle(2, Edge(1, p0, p1), Edge(2, p1, p2), Edge(3, p0, p2))
print(f"Triangle => id: {t.id} (not closed)")
for v in t.vertices:
    print(f"            x:{v.x}, y:{v.y}")
print(f"    Is triangle closed? => {t.closed()}")
# I am a triangle.
# Triangle => id: 1 (closed)
#             x:0.0, y:0.0
#             x:1.0, y:0.0
#             x:0.0, y:1.0
#     Is triangle closed? => True

# I am not a triangle.
# Triangle => id: 2 (not closed)
#             x:0.0, y:0.0
#             x:1.0, y:0.0
#             x:0.0, y:1.0
#     Is triangle closed? => False

Point => x:0.0, y:0.0
True
False

Edge => id: 1, p0: (1.0, 0.0), p1: (0.0, 1.0)

I am a triangle.
Triangle => id: 1 (closed)
            x:0.0, y:0.0
            x:1.0, y:0.0
            x:0.0, y:1.0
    Is triangle closed? => True

I am not a triangle.
Triangle => id: 2 (not closed)
            x:0.0, y:0.0
            x:1.0, y:0.0
            x:0.0, y:1.0
    Is triangle closed? => False
